In [ ]:
!conda install -c conda-forge basemap-data-hires

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import spacy
import regex as re
from datetime import datetime
from tqdm import tqdm
import pickle
import scipy
from mpl_toolkits.basemap import Basemap

customers_dataset = pd.read_csv('olist_customers_dataset.csv')
geolocation_dataset = pd.read_csv('olist_geolocation_dataset.csv')
order_items_dataset = pd.read_csv('olist_order_items_dataset.csv')
order_payments_dataset = pd.read_csv('olist_order_payments_dataset.csv')
order_reviews_dataset = pd.read_csv('olist_order_reviews_dataset.csv')
orders_dataset = pd.read_csv('olist_orders_dataset.csv')
products_dataset = pd.read_csv('olist_products_dataset.csv')
sellers_dataset = pd.read_csv('olist_sellers_dataset.csv')
product_category_name_translation = pd.read_csv('product_category_name_translation.csv')

# Merges datasets that have a similar column
order_items_products = pd.merge(order_items_dataset,products_dataset,on='product_id')
order_items_products_sellers = pd.merge(order_items_products,sellers_dataset,on='seller_id')
two_order_items_products_sellers = pd.merge(order_items_products_sellers,orders_dataset,on='order_id')
two_order_items_products_sellers_customer = pd.merge(two_order_items_products_sellers,customers_dataset,on='customer_id')
two_order_items_products_sellers_customer_reviews = pd.merge(two_order_items_products_sellers_customer,order_reviews_dataset,on='order_id')
final_dataframe = pd.merge(two_order_items_products_sellers_customer_reviews,order_payments_dataset,on='order_id')

mapping = dict(zip(product_category_name_translation['product_category_name'].tolist(),product_category_name_translation['product_category_name_english'].tolist()))
final_dataframe['product_category_name'] = final_dataframe['product_category_name'].map(mapping)

final_dataframe.head(10)

# drops duplicates leaving only unique values and drop Nulls as we cannot use them
final_dataframe = final_dataframe.drop_duplicates(subset=['order_id','order_purchase_timestamp','product_id','customer_unique_id','review_comment_message'])
final_dataframe.shape

final_dataframe.drop(['order_id','product_id','seller_id','customer_unique_id'], axis=1, inplace=True)
final_dataframe.columns

final_dataframe.dropna(subset=['shipping_limit_date','order_purchase_timestamp','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date'], inplace=True)

intermediate_time = final_dataframe['order_delivered_customer_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f0").date()) - final_dataframe['order_purchase_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f0").date())
final_dataframe['purchase-delivery difference'] = intermediate_time.apply(lambda x:x.days)

intermediate_time = final_dataframe['order_estimated_delivery_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f0").date()) - final_dataframe['order_delivered_customer_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f0").date())
final_dataframe['estimated-actual delivery difference'] = intermediate_time.apply(lambda x:x.days)

# Which rows have nulls?
final_dataframe.isnull().sum()

# After finding Null values we need to impute. in place of null texts are used such as review comments and title, we will use NA
final_dataframe['product_category_name'].fillna(value=final_dataframe['product_category_name'].mode()[0], inplace=True)
final_dataframe['product_name_length'].fillna(value=final_dataframe['product_name_length'].mode()[0], inplace=True)
final_dataframe['product_description_length'].fillna(value=final_dataframe['product_description_length'].median(), inplace=True)
final_dataframe['product_photos_qty'].fillna(value=final_dataframe['product_photos_qty'].mode()[0], inplace=True)
final_dataframe['product_weight_g'].fillna(value=final_dataframe['product_weight_g'].mode()[0], inplace=True)
final_dataframe['product_length_cm'].fillna(value=final_dataframe['product_length_cm'].mode()[0], inplace=True)
final_dataframe['product_height_cm'].fillna(value=final_dataframe['product_height_cm'].mode()[0], inplace=True)
final_dataframe['product_width_cm'].fillna(value=final_dataframe['product_width_cm'].mode()[0], inplace=True)
final_dataframe['review_comment_message'].fillna(value='NA', inplace=True)

# We will consider any review_score above 3 as good
final_dataframe['review_score'] = final_dataframe['review_score'].apply(lambda x: 1 if x > 3 else 0)
final_dataframe[['price','freight_value','product_name_length','product_description_length','product_weight_g','product_length_cm', 'product_height_cm', 'product_width_cm']].describe().round(3)

# A new column called price category will be made and this column can be used to categorize if the product was expensive, affordable, or cheap
final_dataframe['price_category'] = final_dataframe['price'].apply(lambda x:'expensive' if x>=139 else ('affordable' if x>=40 and x<139 else 'cheap'))

# What are the top 12 most selling product categories in terms of how frequent people buy them? 
# This Bar chart shows that bed_bath_table is at the top selling over 10k units, health_beauty is second selling over 8k units, and sports_leisure is 3rd selling just under 8k units. 
# On the contrary, Cool Stuff and Garden Tools are at the bottom selling near 4k worth of units.

plt.figure(figsize=(8,8))
idx = final_dataframe['product_category_name'].value_counts()[:10].index
sns.countplot(y=final_dataframe['product_category_name'], order=idx, palette="icefire_r")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Count', fontsize=14)
plt.ylabel('Top categories', fontsize=14)
plt.show()

# Where is the activity happening?

# Shows concentration of where the activity is on a map. According to the map, sales activity is conentrated on the coast where San Paolo is.
latitudes = geolocation_dataset['geolocation_lat'].values
longitudes = geolocation_dataset['geolocation_lng'].values

plt.figure(figsize=(14,9))
m = Basemap(llcrnrlat=-60,llcrnrlon=-85,urcrnrlat=15,urcrnrlon=-20)
m.drawcoastlines()
m.fillcontinents(color='coral')
m.scatter(longitudes,latitudes, zorder=2)
m.drawmapboundary(fill_color='aqua')
m.drawcountries()

plt.show()

# Shows concentration of activity on a bar chart and it happens to be in San Paolo.
# Between 2016-2018 over 40,000 units were sold in this particular state.
# Suggesting that this area is doing fine and we should consider more problem areas.

plt.figure(figsize=(8,7))
idx = final_dataframe['customer_state'].value_counts()[:12].index
sns.countplot(y=final_dataframe['customer_state'], order=idx, palette="icefire_r")
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('Count', fontsize=14)
plt.ylabel('Top categories', fontsize=14)
plt.show()

# What are prices for the products?

# In this bar chart we can see average price per top categories not including freight_value
# where computers is the most expensive at around 1500 real
# and small_appliances_home_oven_coffee are second making up about half of how much computers are.

grp_cat_name = final_dataframe.groupby('product_category_name')
grp_dict = dict(grp_cat_name.mean()['price'].sort_values(ascending=False)[:12])
plt.figure(figsize=(8,8))
sns.barplot(y=list(grp_dict.keys()),x=list(grp_dict.values()), palette="icefire_r")
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('Average price', fontsize=15)
plt.ylabel('Top categories', fontsize=15)
plt.show()

#In this bar chart we account for freight value.
# When taking freight_value into consideration, items such as acs, office_furniture, kitchen_dining_laundry_furniture move up in expenses.

grp_dict = dict((grp_cat_name.mean()['price']+grp_cat_name.mean()['freight_value']).sort_values(ascending=False)[:12])
plt.figure(figsize=(8,8))
sns.barplot(y=list(grp_dict.keys()),x=list(grp_dict.values()), palette="icefire_r")
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('Average price', fontsize=15)
plt.ylabel('Top categories', fontsize=15)
plt.show()

# Which city generates the highest revenue? pianco 
